In [1]:
import requests
import json
import csv
import traceback
import time
import os
import pandas as pd
# from futile.json import json_at
# from futile.file import ThreadSafeWriter
# 请求参数解析
def url_parse(url_str):
    lis=url_str.split('?')
    params=lis[1].split('&')
    dic={}
    for item in params:
        item=item.split('=')
        dic[item[0]]=item[1]
    return dic

header_tag=['content_id','userid','nickname','likes','shareLink']

def get_tab_content(tag_name,url_str):
    ret = {}
    request_num=0
    with open("tag_"+tag_name+".csv",'a+') as csvFile:
        writer=csv.writer(csvFile,delimiter=',')
        writer.writerow(header_tag)
        vedio_num=len(pd.read_csv("tag_"+tag_name+".csv"))
#         vedio_num=0
        while True:
            time.sleep(1)
            if vedio_num>2000:
                break
            try:
                params=url_parse(url_str)
                response = requests.get(url_str,
                    timeout=5,
                    headers={
                        "Authorization": params['sid'],
                        "device_id": params['deviceId'],
                        "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.0.2; Redmi Note 2 MIUI/6.1.29) Resolution/1080*1920 Version/5.19.0 Build/5190110 Device/(Xiaomi;Redmi Note 2)",
                        "shield": "",
                        "Host": "www.xiaohongshu.com",
                        "Connection":"Keep-Alive",
                        "Accept-Encoding": "gzip"
                    })
                
#                 print(url_str)
                res_data=response.json()['data']
                for item in res_data:
                    if item['id'] not in ret.keys():
                        ret[item['id']]=1
                        line=[item['id'],
                             item['user']['userid'],
                             item['user']['nickname'],
                             str(item['likes']),
                             item['share_link']]
                        writer.writerow(line)
                        vedio_num+=1
                        print("{}: 录入{}条".format(tag_name,vedio_num))
                print('{}请求{}次'.format(tag_name,request_num))
                request_num+=1
            except :
                print("{}第{}条 发生错误！".format(tag_name,vedio_num))
                traceback.print_exc()
#                 print(response.json())
                break
    print(tag_name,"Done!")

# get_list(dict_oid["视频"],dict_sign["视频"])

In [2]:
import threading

url_1="https://www.xiaohongshu.com/api/sns/v5/homefeed?oid=homefeed_recommend&cursor_score=&geo=eyJsYXRpdHVkZSI6MzkuOTcxOTc0LCJsb25naXR1ZGUiOjExNi4zMjk1NTF9%0A&platform=Android&deviceId=0fccbfe6-8bbd-33d5-a059-2bb287f2f933&device_fingerprint=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&device_fingerprint1=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&versionName=5.19.0&channel=xiaohongshu&sid=session.1215547881233182228&lang=zh-Hans&t=1531051917&sign=78cef9391de205cc86dd5fb208d4e01c"
# get_tab_content(tag_name="recommend_1",url_str=url_1)

url_2="https://www.xiaohongshu.com/api/sns/v5/homefeed?oid=homefeed_recommend&cursor_score=&geo=eyJsYXRpdHVkZSI6MzkuOTcxOTc0LCJsb25naXR1ZGUiOjExNi4zMjk1NTF9%0A&platform=Android&deviceId=0fccbfe6-8bbd-33d5-a059-2bb287f2f933&device_fingerprint=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&device_fingerprint1=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&versionName=5.19.0&channel=xiaohongshu&sid=session.1215602350880663689&lang=zh-Hans&t=1531107683&sign=2cde1610502713b6f9b4da5f964a0740"
#新用户
url_3="https://www.xiaohongshu.com/api/sns/v5/homefeed?oid=homefeed_recommend&cursor_score=&geo=&platform=Android&deviceId=0598db32-6d98-38aa-bdbb-9a101a1b19a1&device_fingerprint=20180706184416e08e7ddd952ab092117687d072cdd3f2017bdcdc3bd19d87&device_fingerprint1=20180706184416e08e7ddd952ab092117687d072cdd3f2017bdcdc3bd19d87&versionName=5.19.0&channel=Xiaomi&sid=session.1215604349666270009&lang=zh-Hans&t=1531108436&sign=75b8b384a71dd8fe4fefeb1b2b8a5f95"
thd_user2=threading.Thread(target=get_tab_content,args=("recommend_2",url_2,))
thd_user3=threading.Thread(target=get_tab_content,args=("recommend_3",url_3,))

# threads=[thd_user2,thd_user3]
# for _thread in threads:
#     _thread.setDaemon(True)
#     _thread.start()
    

In [3]:
# 获取用户 粉丝  赞+收藏
user_count=[1,2,3]
user_tags=["recommend"]
user_name=0

def get_user_info(account,users,tag):
    '''
            tag tag栏
            account 登录账号
            users 用户列表
    '''
    global user_name
    with open("result/user{}/{}_users.csv".format(account,tag),'a+') as csvFile:
        #  gender 1女，fans 粉丝，follows 关注，liked 喜欢，collect 收藏，
        headers=['userid','nickname','gender','fans','follows','liked&collected']
        writer=csv.writer(csvFile,delimiter=',')
        writer.writerow(headers)
        for user in users:
            try:
                url_str="https://www.xiaohongshu.com/api/sns/v3/user/{}/info?platform=Android&deviceId=0fccbfe6-8bbd-33d5-a059-2bb287f2f933&device_fingerprint=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&device_fingerprint1=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&versionName=5.19.0&channel=xiaohongshu&sid=session.1215602350880663689&lang=zh-Hans&t=1531110409&sign=5932a1e465a76e3066890e35cf69ce3a".format(user)
                params=url_parse(url_str)
                response = requests.get(url_str,
                    timeout=5,
                    headers={
                        "Authorization": params['sid'],
                        "device_id": params['deviceId'],
                        "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.0.2; Redmi Note 2 MIUI/6.1.29) Resolution/1080*1920 Version/5.19.0 Build/5190110 Device/(Xiaomi;Redmi Note 2)",
                        "shield": "",
                        "Host": "www.xiaohongshu.com",
                        "Connection":"Keep-Alive",
                        "Accept-Encoding": "gzip"
                    }
                )
                item=response.json()['data']
#                         'gender','fans','follows','liked','collected'
                line=[item['userid'],
                     item['nickname'],
                     item['gender'],
                     item['fans'],
                     item['follows'],int(item['liked'])+int(item['collected'])]
                
                writer.writerow(line)
                user_name+=1
                print("第{}位用户：".format(user_name))
            except Exception:
                print("第{}条 发生错误！".format(user_name))
                print(str(Exception))
                pass

file_list=["tag_recommend_1.csv","tag_recommend_2","tag_recommend_3"]
# for user in user_count:
#     for tag in user_tags:
#         data_frame=pd.read_csv("tag_"+tag+"_"+str(user)+".csv")
#         users=data_frame['userid'].drop_duplicates()
#         get_user_info(user,users,tag)

In [4]:
# get top 500 users
top_personal=300

top_final=500
# 互动数
top_reactive=20
# 官方推荐数
top_official=10

def getfinal_user():
    lis=[]
    for account in user_count:
        for tag in user_tags:
            _data=pd.read_csv("result/user{}/{}_users.csv".format(account,tag))
            _data=_data.sort_values(by=['fans'],ascending=False)[0:top_personal]
            lis.append(_data)
    final=(pd.concat(lis,ignore_index=True)).drop_duplicates('userid')
    final=final.sort_values(by=['fans'],ascending=False)[0:top_final]
    final.to_csv("final/usersInfo.csv",index=False)
# getfinal_user()
print("Done!")

Done!


In [5]:
# 请求参数解析
def url_parse(url_str):
    lis=url_str.split('?')
    params=lis[1].split('&')
    dic={}
    for item in params:
        item=item.split('=')
        dic[item[0]]=item[1]
    return dic

# 比较参数差异
def url_parames_diff(dic1,dic2):
    return dict.fromkeys([x for x in dic1 if dic1[x]!=dic2[x]])


recommend_old="https://www.xiaohongshu.com/api/sns/v5/homefeed?oid=homefeed_recommend&cursor_score=&geo=eyJsYXRpdHVkZSI6MzkuOTcxODI5LCJsb25naXR1ZGUiOjExNi4zMjk1NDR9%0A&platform=Android&deviceId=0fccbfe6-8bbd-33d5-a059-2bb287f2f933&device_fingerprint=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&device_fingerprint1=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&versionName=5.19.0&channel=xiaohongshu&sid=session.1215535996775781374&lang=zh-Hans&t=1531049412&sign=85edd46bc818bfab3a02d09bc0d6ab68"
recommend_new="https://www.xiaohongshu.com/api/sns/v5/homefeed?oid=homefeed_recommend&cursor_score=&geo=&platform=Android&deviceId=0fccbfe6-8bbd-33d5-a059-2bb287f2f933&device_fingerprint=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&device_fingerprint1=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&versionName=5.19.0&channel=xiaohongshu&sid=session.1215535996775781374&lang=zh-Hans&t=1531049326&sign=6c659a72750a1bdcf868b72b7f73ac9e"

print(url_parames_diff(url_parse(recommend_old),url_parse(recommend_new)))

{'geo': None, 't': None, 'sign': None}


In [15]:
# 官方收录 
import pickle
url_office_judge="https://www.xiaohongshu.com/api/sns/v3/user/{}/info?platform=Android&deviceId=0fccbfe6-8bbd-33d5-a059-2bb287f2f933&device_fingerprint=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&device_fingerprint1=20180708105825f6071809c0879490504603a3da37b230019baed54dc70142&versionName=5.19.0&channel=xiaohongshu&sid=session.1215631826378363046&lang=zh-Hans&t=1531191995&sign=0cdd57bec98fccf9e664ac47c8205081"

def get_ndNums_isOfficial(url_office_judge,userid):
    try:
        params=url_parse(url_office_judge.format(userid))
        response = requests.get(url_office_judge.format(userid),
            timeout=5,
            headers={
                "Authorization": params['sid'],
                "device_id": params['deviceId'],
                "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.0.2; Redmi Note 2 MIUI/6.1.29) Resolution/1080*1920 Version/5.19.0 Build/5190110 Device/(Xiaomi;Redmi Note 2)",
                "shield": "",
                "Host": "www.xiaohongshu.com",
                "Connection":"Keep-Alive",
                "Accept-Encoding": "gzip"
            }
        )
        item=response.json()['data']
        office=item["recommend_info"]
        office_num=0
        if len(office)>0:
            office_num=int(office.replace("篇笔记被官方内容收录",""))
#         isOffical=(False if item["recommend_info"]=="" else True)
        return item['ndiscovery'],office_num
    except Exception:
        print("{} 有错".format(userid))
        pass
    
users=pd.read_csv("final/usersInfo.csv")['userid']
list_users_nums_office=[]
user_iter=0
for user in users:
    user_iter+=1
#     if user_iter==3:
#         break
    nums,isOffice=get_ndNums_isOfficial(url_office_judge,user)
    list_users_nums_office.append((user,nums,isOffice))
    print(user_iter)
    
with open("final/users_nums_isOfficial.pkl",'wb') as pk:
    pickle.dump(list_users_nums_office,pk)

print("Done!")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
# 获取top20

import time
import traceback
import pickle

#全局 保存用户 notesid
USERID_NOTEIDS=[]

format_time = '2018-04-9'
early_time = time.mktime(time.strptime(format_time, "%Y-%m-%d"))
print(early_time)

def get_data(url_str,_params=('1',)):
    try:
        params=url_parse(url_str)
        response = requests.get(url_str,
            timeout=10,
            headers={
                "Authorization":"",
                "device_id":"" ,
                "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.0.2; Redmi Note 2 MIUI/6.1.29) Resolution/1080*1920 Version/5.19.0 Build/5190110 Device/(Xiaomi;Redmi Note 2)",
                "shield": "",
                "Host": "www.xiaohongshu.com",
                "Connection":"Keep-Alive",
                "Accept-Encoding": "gzip"
            }
        )
        data=response.json()["data"]
        return data
    except Exception:
        print("{} 发生错误！".format(_params[0]))
#         print(url_str)
#         traceback.print_exc()

# 获取notes 长度
# url_user="https://www.xiaohongshu.com/web_api/sns/v1/user/{}/board?page=1"
url_user="https://www.xiaohongshu.com/api/sns/v3/user/{}/info?platform=Android&deviceId=0598db32-6d98-38aa-bdbb-9a101a1b19a1&device_fingerprint=20180706184416e08e7ddd952ab092117687d072cdd3f2017bdcdc3bd19d87&device_fingerprint1=20180706184416e08e7ddd952ab092117687d072cdd3f2017bdcdc3bd19d87&versionName=5.19.0&channel=Xiaomi&sid=session.1215604349666270009&lang=zh-Hans&t=1531189557&sign=f7734dbe55e63343cdc4c1c320bf5bdf"
def get_note_num(url_user,userid):
#     list_users=pd.read_csv("final/usersInfo.csv")['userid']
    print("用户：{}".format(userid))
    print(url_user.format(userid))
    try_nums=0
    for i in range(2):
        try:
            data=get_data(url_user.format(userid))["ndiscovery"]
            return data
        except:
            if i==1:
                writer_log.writerow(userid)
            pass
        

# 获取笔记详情--互动数
url_notedetail="https://www.xiaohongshu.com/api/sns/v9/note/{}?filter=explore&platform=Android&deviceId=0598db32-6d98-38aa-bdbb-9a101a1b19a1&device_fingerprint=20180706184416e08e7ddd952ab092117687d072cdd3f2017bdcdc3bd19d87&device_fingerprint1=20180706184416e08e7ddd952ab092117687d072cdd3f2017bdcdc3bd19d87&versionName=5.19.0&channel=Xiaomi&sid=session.1215604349666270009&lang=zh-Hans&t=1531188795&sign=a62e6b90fec1def3c67719e0182f3234"
def get_note_active_num(url_notedetail,noteid,more_than_num=True):
    detail=get_data(url_notedetail.format(noteid))
    try:
        post_time=detail['post_time']
        active_num=detail['comments']+detail['fav_count']+detail['likes']
        if post_time<int(early_time) and more_than_num:
            return -1
        else:
            return active_num
    except Exception:
        print(str(Exception))
    
#获取note id 指定时间内的
url_note="https://www.xiaohongshu.com/web_api/sns/v1/user/{}/note?page={}&page_size=100"
def get_user_notes(url_detail,userid,TOP_K=20):
    nums=get_note_num(url_user,userid)
    notes=[]
    for page in range(1,int(nums/100+2)):
        print("笔记页面请求")
        list_note=get_data(url_note.format(userid,page))
        try:
            if len(list_note)==0:
                continue
            for note in list_note:
                notes.append(note['id'])
        except:
            pass
            
    USERID_NOTEIDS.append({userid:notes})
    dict_note={}
    for note in notes:
        try:
            active_num=get_note_active_num(url_notedetail,note,nums>TOP_K)
            if active_num>0:
                dict_note[note]=active_num
            print("核对中...")
        except:
            pass
            
        
    # 按键值降序
    dict_note=sorted(dict_note.items(),key=lambda d:d[1], reverse=True)
    return dict_note

def notes2file(users,writer,TOPK=20):
    nth=0
    for user in users:
        # 获取前K笔记
        nth+=1
        print("第 {} 位用户".format(nth))
        if nth<68:
            continue
        items=get_user_notes(url_note,user)
        line=[user]
        line.extend(["https://www.xiaohongshu.com/discovery/item/{}".format(i[0]) for i in items])
        if len(line)>TOPK+1:
            line=line[0:TOPK+1]
        writer.writerow(line)
    writer
# header
head_20=['userid']
for i in range(1,top_reactive+1):
    head_20.append("link{}".format(i))
# errors logs
writer_log=csv.writer(open("final/err_userid.csv",'a+'))

with open("final/TOP_20.csv",'a+') as topCsv:
    writer=csv.writer(topCsv,delimiter=',')
#     writer.writerow(head_20)
    users=pd.read_csv("final/usersInfo.csv")['userid']
    notes2file(users,writer,top_reactive)
    
# print(get_user_notes(url_note,'5ad577e6e8ac2b2ad36ceb77'))


with open("final/users_noteIds.pkl",'wb') as pk:
    pickle.dump(USERID_NOTEIDS,pk)

    

In [54]:
# user_null_note={}
# with open("final/users_noteIds.pkl",'wb') as pkfile:
#     pickle.dump(user_null_note,pkfile)
# print("Done")

Done


In [57]:
# 获取 每个用户----noteid 
url_noteid="https://www.xiaohongshu.com/web_api/sns/v1/user/{}/note?page={}&page_size=50"

user_nums=pickle.load(open("final/users_nums_isOfficial.pkl",'rb'))

USERS_NOTEIDS=pickle.load(open("final/users_noteIds.pkl",'rb'))
# print(USERS_NOTEIDS.keys)
# get noteid_list of each user
print(USERS_NOTEIDS)
#
run_nums=0
for user,nd_nums,office_nums in user_nums:
    run_nums+=1
    print(run_nums)
    if user in USERS_NOTEIDS.keys():
        continue
    lis_noteid=[]
    if run_nums==10:
        break
    for page in range(1,int(nd_nums/100+2)):
        loops=1
        while True:
            try:
                params=url_parse(url_noteid.format(user,page))
#                 proxies={
#                 'http': '10.3.14.61:8080',
#                 'https': '10.3.14.61:3128'},
                response = requests.get(url_noteid.format(user,page),
                timeout=10)
                items=response.json()['data']
                for item in items:
                    lis_noteid.append(item['id'])
                print(run_nums)
                break
            except Exception:
                if loops>8:
                    break
                else:
                    loops+=1
                    print("{} 有错,重试一次".format(user))
                    pass
    if len(lis_noteid)>0:
        USERS_NOTEIDS[user]=lis_noteid

with open("final/USERS_NOTEIDS.pkl",'wb') as pk:
    pickle.dump(USERS_NOTEIDS,pk)

# te=pickle.load(open("final/USERS_NOTEIDS.pkl",'rb'))
# print(te)

{'58fd96255e87e71f726905f1': ['5b42e06607ef1c7c87dbdf2d', '5b403d9f672e1417e1f63307', '5b3f327607ef1c7c9bdba213', '5b3b170d07ef1c562767032e', '5b3778de672e1430d5386a81', '5b363671910cf66734b02dc7', '5b309dcc07ef1c1d01434a05', '5b2e418a910cf67e693c2dbb', '5b2657d9aac7cb70ef60fb0c', '5b23c28a07ef1c4194ce572a', '5b1f8fa97ee0a97c50e60242', '5b1d1160aac7cb59f8321c22', '5b166eb6aac7cb5a04320ff2', '5b13ed477ee0a97c54e5aa73', '5b1142386b50755d2a2c5316', '5b0fe77fa7c9b865173869a1', '5b0ea2146e88937fa870fb66', '5b0bfd3efb2a3674b2625c4c', '5b09524a4b884554ce0a9341', '5b0658eb6e88935bfa9940d2', '5b03f9c64b88450af2e25549', '5b02c3c2937426450015c051', '5afeec186b50752c8aa7ae15', '5afcf63a6e8893240c33ec07', '5afac342278bd32b8431a706', '5af9889614b84e6160c090af', '5af4fa4407ef1c03d670216d', '5af2ba9026c62425dabf2b36', '5af178f66e889324063385b4', '5af016424df9834222b5f68f', '5aeed407aac7cb3c1e784cb4', '5aebe7c1aac7cb3c1c787c8e', '5ae99cbd672e146fd870724c', '5ae6d54207ef1c03e46fd69c', '5ae41228c4463f6a9

11
12
12
12
12
12
12
12
13
13
14
14
14
14
15
15
15
15
16
16
17
17
17
17
17
17
17
17
17
18
18
18
18
18
18
19
19
20
20
21
21
22
22
22
22
23
23
23
23
24
24
24
24
25
25
25
25
25
26
26
26
26
26
26
27
27
27
27
28
28
28
29
29
29
29
29
29
29
30
30
30
30
31
31
31
31
31
32
32
32
33
33
33
33
33
33
33
33
34
34
34
34
35
35
35
36
36
36
36
36
37
37
37
37
38
38
38
38
38
39
39
39
40
40
41
41
42
42
42
43
43
43
43
43
43
44
44
45
45
45
45
45
46
46
46
46
46
46
47
47
48
48
48
48
48
49
49
49
50
50
50
50
51
51
51
52
52
52
52
53
53
53
53
54
54
54
54
55
55
55
55
56
56
56
56
56
56
56
57
57
57
58
58
58
59
59
60
60
61
61
61
61
62
62
63
63
63
64
64
64
64
64
65
65
65
65
65
65
65
66
66
66
66
67
67
67
68
68
69
69
70
70
71
71
71
72
72
73
73
73
73
74
74
75
75
76
76
77
77
77
77
78
78
78
78
79
79
79
79
79
79
79
79
79
80
80
80
81
81
81
81
82
82
82
82
82
82
83
83
83
83
84
84
84
84
85
85
85
86
86
86
86
87
87
87
87
87
88
88
89
89
89
90
90
90
90
91
91
91
92
92
92
93
93
93
93
93
94
94
94
95
95
95
95
95
95
95
95
95
95
96
96
97
9

In [8]:
import bs4
import csv
import os
import time
import pickle
import requests
import traceback
import time

# 最早笔记时间
format_time = '2018-04-9'
early_time = time.mktime(time.strptime(format_time, "%Y-%m-%d"))
early_time=int(early_time)

# 目标笔记数
K_note=20

# 笔记详情url(赞 喜欢 评论 时间)
url_note="https://www.xiaohongshu.com/api/sns/v9/note/{}?filter=explore&platform=Android&deviceId=0598db32-6d98-38aa-bdbb-9a101a1b19a1&device_fingerprint=20180706184416e08e7ddd952ab092117687d072cdd3f2017bdcdc3bd19d87&device_fingerprint1=20180706184416e08e7ddd952ab092117687d072cdd3f2017bdcdc3bd19d87&versionName=5.19.0&channel=Xiaomi&sid=session.1215604349666270009&lang=zh-Hans&t=1531188795&sign=a62e6b90fec1def3c67719e0182f3234"

# 获取笔记互动数 并讲前K个写入文件
def get_note_detail(writer,users_notes,url_note,topk):
    run_nums=0
    for user in users_notes.keys():
        tmp_result=[user]
        dict_notes={}
        run_nums+=1
        notes=users_notes[user]
        infos={}
        print("{}号用户".format(run_nums))
        #  已抓取
        if run_nums<=0:
            continue
        if run_nums>=2:
            break
        isWrite=True
        for note in notes:
            try:
#                 response = requests.get(url_note.format(note),timeout=10,
#                                         proxies={
#                                             'http': '10.4.18.170:3128',
#                                             'https': '10.4.18.169:3128'},
#                                         headers={"Authorization": "session.1215604349666270009",
#                                             "device_id": "0598db32-6d98-38aa-bdbb-9a101a1b19a1",
#                                             "User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.0.2; Redmi Note 2 MIUI/6.1.29) Resolution/1080*1920 Version/5.19.0 Build/5190110 Device/(Xiaomi;Redmi Note 2)",
#                                             "shield": "",
#                                             "Use-IP-Pool": "001",
#                                             "Host": "www.xiaohongshu.com",
#                                             "Connection":"Keep-Alive",
#                                             "Accept-Encoding": "gzip"
#                                                 })
                time.sleep(1)
                response = requests.get(url_note.format(note),timeout=10)
                #  print(response.text)
                detail=response.json()['data']
                infos[note]=detail
                # 获取互动数
                act_nums=detail['fav_count']+detail['comments']+detail['likes']
                post_time=detail['post_time']
                if post_time>early_time:
                    dict_notes[note]=act_nums
                else:
                    break
                print("parsing...")
            except:
                print("{}号用户笔记：{}出错".format(run_nums,note))
                # traceback.print_exc()
                if response.status_code ==403:
                    print("网络错误：{}".format(403))
                    isWrite=False
                    break
                pass       
        # 取前K个
        if not os.path.exists("notesInfo/{}.pkl".format(user)):
            pickle.dump(infos,open("notesInfo/{}.pkl".format(user),'wb'))
        dict_notes=sorted(dict_notes.items(),key=lambda d:d[1], reverse=True)
        note_count=1
        for tmp_key,tmp_value in dict_notes:
            if note_count>topk:
                break
            tmp_result.append("www.xiaohongshu.com/discovery/item/{}".format(tmp_key))
            note_count+=1
        if isWrite:
            writer.writerow(tmp_result)  

# 加载用户笔记列表["userid":["noteid"]]
users_notes=pickle.load(open("final/users_noteIds.pkl",'rb'))

# 目标前20文件 header
head_20=['userid']
for i in range(1,K_note+1):
    head_20.append("link{}".format(i))

# 存入TOP——20文件
with open("final/RE_TOP_20.csv",'a+') as topCsv:
    writer=csv.writer(topCsv,delimiter=',')
#     writer.writerow(head_20)
    get_note_detail(writer,users_notes,url_note,K_note)
    


1号用户
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
parsing...
1号用户笔记：5b0bfd3efb2a3674b2625c4c出错
网络错误：403
2号用户


In [39]:
list_oid=[{"name":"推荐","oid":"homefeed_recommend"},
        {"name":"视频","oid":"homefeed.video"},
        {"name":"世界杯","oid":"homefeed.costum"},
        {"name":"旅行","oid":"homefeed.travel_v2"},
        {"name":"男士穿搭","oid":"homefeed.mens_fashion_v2"},
        {"name":"健身","oid":"homefeed.fitness_v2"},
        {"name":"读书","oid":"homefeed.books_v2"},
        {"name":"美食","oid":"homefeed.food_v2"},
        {"name":"明星","oid":"homefeed.celebrities_v2"},
        {"name":"时尚","oid":"homefeed.fashion_v2"},
        {"name":"护肤","oid":"homefeed.skincare_v2"},
        {"name":"家居","oid":"homefeed.home_v2"},
        {"name":"数码","oid":"homefeed.digital_v2"},
        {"name":"宠物","oid":"homefeed.pets_v2"},
        {"name":"萌娃","oid":"homefeed.baby_v2"},
        {"name":"母婴","oid":"homefeed.maternity_v2"},
        {"name":"婚礼","oid":"homefeed.weddings_v2"},
        {"name":"彩妆","oid":"homefeed.cosmetics_v2"},
        {"name":"音乐","oid":"homefeed.music_v2"}]
dict_oid={}
for item in list_oid:
    dict_oid[item["name"]]=item["oid"]
dic_test={'name': '推荐'}
print([key for key in dic_test.keys()][0])

name
